In [1]:
# %load Extract_MovieLens_Data.py


# # This notebook is to extract data from Movie Lens
# * The data contents are explained in http://files.grouplens.org/papers/ml-1m-README.txt
# 
# ## users.dat
# 
# UserID::Gender::Age::Occupation::Zip-code
# - Gender is denoted by a "M" for male and "F" for female
# - Age is chosen from the following ranges:
# 
# 	*  1:  "Under 18"
# 	* 18:  "18-24"
# 	* 25:  "25-34"
# 	* 35:  "35-44"
# 	* 45:  "45-49"
# 	* 50:  "50-55"
# 	* 56:  "56+"
# 
# - Occupation is chosen from the following choices:
# 
# 	*  0:  "other" or not specified
# 	*  1:  "academic/educator"
# 	*  2:  "artist"
# 	*  3:  "clerical/admin"
# 	*  4:  "college/grad student"
# 	*  5:  "customer service"
# 	*  6:  "doctor/health care"
# 	*  7:  "executive/managerial"
# 	*  8:  "farmer"
# 	*  9:  "homemaker"
# 	* 10:  "K-12 student"
# 	* 11:  "lawyer"
# 	* 12:  "programmer"
# 	* 13:  "retired"
# 	* 14:  "sales/marketing"
# 	* 15:  "scientist"
# 	* 16:  "self-employed"
# 	* 17:  "technician/engineer"
# 	* 18:  "tradesman/craftsman"
# 	* 19:  "unemployed"
# 	* 20:  "writer"
# 
# ## movies.dat
# MovieID::Title::Genres
# 
# ## ratings.dat
# UserID::MovieID::Rating::Timestamp

# In[1]:

import pandas as pd

# In[3]:

users_df = pd.read_csv("./ml-1m/users.dat"
                    , sep='::'
                    , header=None
                    , names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])
ocupation_codes = {'ocupation_code': [x for x in range(21)]
                   , 'Occupation_name': ["other or not specified", "academic/educator", "artist"
                                  , "clerical/admin", "college/grad student", "customer service"
                                  , "doctor/health care", "executive/managerial", "farmer"
                                  , "homemaker", "K-12 student", "lawyer", "programmer", "retired"
                                  , "sales/marketing" ,"scientist", "self-employed", "technician/engineer"
                                  , "tradesman/craftsman", "unemployed", "writer"]
                  }
ocupation_codes = pd.DataFrame(ocupation_codes)
users_df = users_df.merge(ocupation_codes, left_on=["Occupation"], right_on=["ocupation_code"], how='left')
users_df = users_df.drop(["Occupation", "ocupation_code"], axis=1).rename(columns={'Occupation_name': 'Occupation'})


# In[4]:

movies_org_df = pd.read_csv("./ml-1m/movies.dat"
                            , sep='::'
                            , header=None
                            , names=["MovieID", "Title", "Genres"])
rows = []
for _, row in movies_org_df.iterrows():
    for gen in row.Genres.split('|'):
        rows.append([row['MovieID'], row['Title'], gen])
movies_df = pd.DataFrame(rows, columns=movies_org_df.columns)


# In[5]:

ratings_df = pd.read_csv("./ml-1m/ratings.dat"
                         , sep='::'
                         , header=None
                         , names=["UserID", "MovieID", "Rating", "Timestamp"])
ratings_df['rating_dt'] = pd.to_datetime(ratings_df['Timestamp'],unit='s')

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:60: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:79: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:92: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [2]:
print(users_df.columns)
print(movies_df.columns)
print(ratings_df.columns)

Index(['UserID', 'Gender', 'Age', 'Zip-code', 'Occupation'], dtype='object')
Index(['MovieID', 'Title', 'Genres'], dtype='object')
Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'rating_dt'], dtype='object')


In [3]:
df = ratings_df.merge(users_df, on=['UserID'], how='left').merge(movies_df, on=['MovieID'], how='left')

In [ ]:
df.columns

In [9]:
import datetime as dt
import numpy as np

In [ ]:
ratings_2001_Q3 = df[(df.rating_dt < dt.datetime(2000, 10, 1)) & (df.rating_dt >= dt.datetime(2000, 7, 1))]

In [ ]:
ratings_2001_Q3[["Rating_mean", "Age_mean"]] = ratings_2001_Q3.groupby(["Genres", "Gender", "Occupation"])[['Rating', 'Age']] \
.transform(np.mean).rename(columns={
    "Rating": "Rating_mean"
    , "Age": "Age_mean"
})

In [ ]:
a = ratings_2001_Q3.groupby(["Genres", "Gender", "Occupation"])[['Rating', 'Age']].agg([np.mean, np.std])
a = a.reset_index()
# a.columns = a.columns.get_level_values(0)
a.columns.values
a.columns = [x[:-1] if x.endswith('_') else x for x in ['_'.join(x) for x in a.columns.values]]
a

In [ ]:
#df["Rating_mean"] = df.groupby(["Occupation", "Genres"])['Rating'].transform(np.mean)
#df.sort_values(by="Rating_mean", ascending=False).groupby(["Occupation"]).head(5)
# df.groupby(["Occupation", "Genres"])['Rating'].agg(np.mean).reset_index().rename(columns={"Rating": "Rating_avg"}).sort_values(by="Rating_avg", ascending=False).groupby(["Occupation"]).head(5).sort_values(by=["Occupation", "Rating_avg"])
df.groupby(["Occupation", "Genres"])['Rating'].agg(np.mean).reset_index().rename(columns={"Rating": "Rating_avg"}).sort_values(by="Rating_avg", ascending=False).groupby(["Occupation"]).head(5).sort_values(by=["Occupation", "Rating_avg"])

In [ ]:
# Find top 5 ratings for each occupation
#df = df.groupby(['Occupation', 'Genres']).agg({'Rating': np.mean}).reset_index()
df.assign(Rating_avg=df.groupby(['Occupation', 'Genres'])[['Rating']].transform(np.mean))[['Occupation', 'Genres', 'Rating_avg']] \
.drop_duplicates() \
.sort_values(by="Rating_avg", ascending=False) \
.groupby(["Occupation"]).head(5).sort_values(by=["Occupation", "Rating_avg"])

In [ ]:
# df.groupby(["Occupation", "Genres"]).transform(lambda x : x if print(type(x)) else x)
# df.groupby(["Occupation", "Genres"])['Rating'].transform(lambda x : x if print(len(x)) else x)

In [27]:
# df.groupby(["Occupation", "Genres"])['Rating'].transform(lambda x: x.mean())
# df.groupby(["Occupation", "Genres"]).transform(lambda x: x.mean())
# df.groupby(["Occupation", "Genres"])['Rating'].apply(lambda x: x.mean())
# df.groupby(["Occupation", "Genres"])['Rating'].agg(lambda x: x.mean())

UserID      MovieID    Rating  \
Occupation        Genres                                            
K-12 student      Action       2532.905884  1767.276084  3.497116   
                  Adventure    2538.595813  1492.607354  3.425658   
                  Animation    2576.597967  1773.637246  3.463956   
                  Children's   2586.375772  1518.847479  3.220679   
                  Comedy       2545.116746  1891.971896  3.497200   
                  Crime        2429.580812  1671.200000  3.687085   
                  Documentary  2478.510204  2253.989796  3.581633   
                  Drama        2433.251333  1907.594500  3.782167   
                  Fantasy      2653.578824  1560.192941  3.298039   
                  Film-Noir    2389.336170  1751.102128  4.212766   
                  Horror       2634.205774  2118.437795  3.237795   
                  Musical      2565.968794  1405.688652  3.556738   
                  Mystery      2486.904372  1680.539617  3.636612   
                  Romance      2588.904682  1561.089298  3.624415   
                  Sci-Fi       2563.380977  1792.474059  3.443795   
                  Thriller     2520.013770  1902.263533  3.554131   
                  War          2519.782671  1449.006498  3.880144   
                  Western      2405.880000  1801.423333  3.513333   
academic/educator Action       3390.284421  1779.970064  3.392063   
                  Adventure    3393.275771  1636.572589  3.424278   
                  Animation    3207.027761  1789.463603  3.693399   
                  Children's   3320.729464  1572.640179  3.459286   
                  Comedy       3316.762363  1948.195532  3.519583   
                  Crime        3388.509801  1648.486247  3.659342   
                  Documentary  3251.521411  1606.926952  3.984887   
                  Drama        3333.342690  1803.916621  3.754293   
                  Fantasy      3263.636265  1742.813970  3.456388   
                  Film-Noir    3589.960043  1673.835313  4.082613   
                  Horror       3412.892695  2074.779090  3.104548   
                  Musical      3278.801924  1592.500130  3.701586   
...                                    ...          ...       ...   
unemployed        Documentary  2523.280303  1760.280303  3.727273   
                  Drama        2541.939486  1768.673143  3.619308   
                  Fantasy      3012.820034  1788.361630  3.230900   
                  Film-Noir    2428.537037  1737.092593  4.044444   
                  Horror       2741.346204  2110.849476  3.136126   
                  Musical      2927.243433  1619.271454  3.360771   
                  Mystery      2383.503322  1740.543189  3.579734   
                  Romance      2727.904666  1524.278475  3.451581   
                  Sci-Fi       2534.441553  1828.035109  3.330855   
                  Thriller     2392.388555  1699.196044  3.403038   
                  War          2614.687356  1459.390805  3.693103   
                  Western      2704.679715  1876.291815  3.501779   
writer            Action       2927.703535  1798.280230  3.339971   
                  Adventure    2980.327448  1674.252082  3.346110   
                  Animation    2929.141994  1808.995846  3.699773   
                  Children's   2933.299930  1587.369469  3.351440   
                  Comedy       2865.536393  1972.912412  3.423004   
                  Crime        2906.468133  1678.281613  3.651598   
                  Documentary  2972.449198  1794.600267  3.967914   
                  Drama        2789.416706  1840.595350  3.653348   
                  Fantasy      2959.647895  1778.875789  3.338421   
                  Film-Noir    3125.253835  1724.979777  4.104603   
                  Horror       2837.532659  2075.373965  3.201472   
                  Musical      3031.142123  1590.152773  3.669849   
                  Mystery      2921.540185  1717.306414  3.605100   
                  Romance      281

In [17]:
df.groupby(["Occupation", "Genres"]).transform(lambda x: x if print(len(x)) else x)

6067
6067
3726
2164
3888
9465
1355
98
6000
1275
235
1905
1410
732
2990
3932
4212
1385
300
18673
10215
3242
5600
29949
6326
794
34769
2763
1852
5079
3847
3730
14433
11795
14723
6006
1715
11277
5986
2163
3375
17982
3957
553
19273
1760
1000
3676
2387
2026
7824
7110
8683
3072
973
7078
3700
1321
2226
11870
2469
285
11533
1043
677
2210
1518
1356
5210
4022
5922
1950
659
35152
17910
7036
11264
48672
10772
970
43546
5225
1770
10383
5442
4740
18913
20352
25302
8205
2091
6493
3228
974
1597
7908
1734
97
6369
934
329
2390
888
718
2649
3789
4238
1348
546
8601
4446
1565
2596
13148
2800
302
14332
1216
705
2576
1646
1469
5971
5146
6781
2572
717
28863
14138
3451
6251
35784
8738
707
38888
3664
1865
7479
3881
4270
15453
16316
20654
7981
2695
868
528
134
261
939
198
10
835
152
41
182
100
82
373
550
497
164
51
2098
1299
693
1281
5065
699
35
4152
409
130
542
694
391
2681
1065
1710
664
191
4793
2563
726
1209
7526
1795
191
7832
610
537
1249
874
920
2989
2776
3632
1585
535
32957
16919
5583
9572
46500
10460
1155

,UserID,MovieID,Rating,Timestamp,rating_dt,Gender,Age,Zip-code,Occupation,Title,Genres
0,1,1193,5,978300760,2000-12-31 22:12:40,F,1,48067,K-12 student,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,K-12 student,James and the Giant Peach (1996),Animation
2,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,K-12 student,James and the Giant Peach (1996),Children's
3,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,K-12 student,James and the Giant Peach (1996),Musical
4,1,914,3,978301968,2000-12-31 22:32:48,F,1,48067,K-12 student,My Fair Lady (1964),Musical
5,1,914,3,978301968,2000-12-31 22:32:48,F,1,48067,K-12 student,My Fair Lady (1964),Romance
6,1,3408,4,978300275,2000-12-31 22:04:35,F,1,48067,K-12 student,Erin Brockovich (2000),Drama
7,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,K-12 student,"Bug's Life, A (1998)",Animation
8,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,K-12 student,"Bug's Life, A (1998)",Children's
9,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,K-12 student,"Bug's Life, A (1998)",Comedy


In [13]:
df.assign(max_ratings = ratings_df.groupby("MovieID")['Rating'].transform(np.max))

,UserID,MovieID,Rating,Timestamp,rating_dt,Gender,Age,Zip-code,Occupation,Title,Genres,max_ratings
0,1,1193,5,978300760,2000-12-31 22:12:40,F,1,48067,K-12 student,One Flew Over the Cuckoo's Nest (1975),Drama,5.0
1,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,K-12 student,James and the Giant Peach (1996),Animation,5.0
2,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,K-12 student,James and the Giant Peach (1996),Children's,5.0
3,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,K-12 student,James and the Giant Peach (1996),Musical,5.0
4,1,914,3,978301968,2000-12-31 22:32:48,F,1,48067,K-12 student,My Fair Lady (1964),Musical,5.0
5,1,914,3,978301968,2000-12-31 22:32:48,F,1,48067,K-12 student,My Fair Lady (1964),Romance,5.0
6,1,3408,4,978300275,2000-12-31 22:04:35,F,1,48067,K-12 student,Erin Brockovich (2000),Drama,5.0
7,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,K-12 student,"Bug's Life, A (1998)",Animation,5.0
8,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,K-12 student,"Bug's Life, A (1998)",Children's,5.0
9,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,K-12 student,"Bug's Life, A (1998)",Comedy,5.0
